# 104 Spark - Movielens

The goal of this lab is to run some analysis on a different dataset, [MovieLens](https://grouplens.org/datasets/movielens/), on AWS.

- [Spark programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
- [RDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/RDD.html)
- [PairRDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/PairRDDFunctions.html)

**Download the dataset** from [here](https://big.csr.unibo.it/downloads/bigdata/ml-dataset.zip), unzip it and put it in the ```datasets/big``` folder.

- ml-movies.csv (<u>movieId</u>:Long, title:String, genres:String) 
    - genres are separated by pipelines  (e.g., "comedy|drama|action")
    - each movie is associated with many ratings

- ml-ratings.csv (<u>userId</u>:Long, <u>movieId</u>:Long, rating:Double, year:Int)
    - each rating is associated with many tags
    - ml-ratings-sample.csv is a small sample of ml-ratings.csv, useful for developing
- ml-tags.csv (<u>userId</u>:Long, <u>movieId</u>:Long, <u>tag</u>:String, year:Int) 

In [1]:
import org.apache.spark

Intitializing Scala interpreter ...

Spark Web UI available at http://lab42-04-01.campusfc.dir.unibo.it:4040
SparkContext available as 'sc' (version = 3.5.2, master = local[*], app id = local-1731488324655)
SparkSession available as 'spark'


import org.apache.spark


In [2]:
val path_to_datasets = "../../../../datasets/big/"

val path_ml_movies = path_to_datasets + "ml-movies.csv"
val path_ml_ratings = path_to_datasets + "ml-ratings-sample.csv"
val path_ml_tags = path_to_datasets + "ml-tags.csv"

path_to_datasets: String = ../../../../datasets/big/
path_ml_movies: String = ../../../../datasets/big/ml-movies.csv
path_ml_ratings: String = ../../../../datasets/big/ml-ratings-sample.csv
path_ml_tags: String = ../../../../datasets/big/ml-tags.csv


In [3]:
import java.util.Calendar
import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner

object MovieLensParser {

  val noGenresListed = "(no genres listed)"
  val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
  val pipeRegex = "\\|(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
  val quotes = "\""
  
  /** Convert from timestamp (String) to year (Int) */
  def yearFromTimestamp(timestamp: String): Int = {
    val cal = Calendar.getInstance()
    cal.setTimeInMillis(timestamp.trim.toLong * 1000L)
    cal.get(Calendar.YEAR)
  }

  /** Function to parse movie records
   *
   *  @param line line that has to be parsed
   *  @return tuple containing movieId, title and genres, none in case of input errors
   */
  def parseMovieLine(line: String): Option[(Long, String, String)] = {
    try {
      val input = line.split(commaRegex)
      var title = input(1).trim
      title = if(title.startsWith(quotes)) title.substring(1) else title
      title = if(title.endsWith(quotes)) title.substring(0, title.length - 1) else title
      Some(input(0).trim.toLong, title, input(2).trim)
    } catch {
      case _: Exception => None
    }
  }

  /** Function to parse rating records
   *
   *  @param line line that has to be parsed
   *  @return tuple containing userId, movieId, rating, and year none in case of input errors
   */
  def parseRatingLine(line: String): Option[(Long, Long, Double, Int)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim.toLong, input(1).trim.toLong, input(2).trim.toDouble, yearFromTimestamp(input(3)))
    } catch {
      case _: Exception => None
    }
  }

  /** Function to parse tag records
   *
   *  @param line line that has to be parsed
   *  @return tuple containing userId, movieId, tag, and year, none in case of input errors
   */
  def parseTagLine(line: String) : Option[(Long, Long, String, Int)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim.toLong, input(1).trim.toLong, input(2), yearFromTimestamp(input(3)))
    } catch {
      case _: Exception => None
    }
  }

}

import java.util.Calendar
import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner
defined object MovieLensParser


In [4]:
val rddMovies = sc.textFile(path_ml_movies).flatMap(MovieLensParser.parseMovieLine)
val rddRatings = sc.textFile(path_ml_ratings).flatMap(MovieLensParser.parseRatingLine)
val rddTags = sc.textFile(path_ml_tags).flatMap(MovieLensParser.parseTagLine)

rddMovies: org.apache.spark.rdd.RDD[(Long, String, String)] = MapPartitionsRDD[2] at flatMap at <console>:32
rddRatings: org.apache.spark.rdd.RDD[(Long, Long, Double, Int)] = MapPartitionsRDD[5] at flatMap at <console>:33
rddTags: org.apache.spark.rdd.RDD[(Long, Long, String, Int)] = MapPartitionsRDD[8] at flatMap at <console>:34


## 104-1 Datasets exploration

Cache the datasets and answer the following questions:

- How many (distinct) users, movies, ratings, and tags?
- How many (distinct) genres?
- On average, how many ratings per user?
- On average, how many ratings per movie?
- On average, how many genres per movie?
- What is the range of ratings?
- Which years? (print an ordered list)
- On average, how many ratings per year?

Try these locally as "extra" exercises; solutions will be published later on.

In [5]:
val rddMoviesCached = rddMovies.cache()
val rddRatingsCached = rddRatings.cache()
val rddTagsCached = rddTags.cache()

rddMoviesCached: rddMovies.type = MapPartitionsRDD[2] at flatMap at <console>:32
rddRatingsCached: rddRatings.type = MapPartitionsRDD[5] at flatMap at <console>:33
rddTagsCached: rddTags.type = MapPartitionsRDD[8] at flatMap at <console>:34


## 104-2 Compute the average rating for each movie

- Export the result to a file
- Do not start from cached RDDs
- Evaluate:
  - Join-and-Aggregate
  - Aggregate-and-Join
  - Aggregate-and-BroadcastJoin

In [6]:
val path_output_avgRatPerMovie = "../../../../output/avgRatPerMovie"
// rdd.coalesce(1).toDF().write.format("csv").mode(SaveMode.Overwrite).save(path_output_avgRatPerMovie)

sc.getPersistentRDDs.foreach(_._2.unpersist())

path_output_avgRatPerMovie: String = ../../../../output/avgRatPerMovie


### Join-and-Aggregate

In [8]:
val rddMoviesKV = rddMovies.map(x => (x._1,x._2))
val avgRatPerMovie = rddRatings.
    map(x => ((x._2),(x._3))).
    join(rddMoviesKV).
    map({case (m,(r,t)) => ((m,t),r)}).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).
    map({case ((m,t),(sum,cnt)) => (m, t, sum/cnt, cnt)}).
    coalesce(1).
    toDF().write.format("csv").mode(SaveMode.Overwrite).save(path_output_avgRatPerMovie)

rddMoviesKV: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[21] at map at <console>:31
avgRatPerMovie: Unit = ()


### Aggregate-and-Join

In [9]:
val rddMoviesKV = rddMovies.map(x => (x._1,x._2))
val avgRatPerMovie = rddRatings.
    map(x => (x._2,x._3)).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).
    mapValues({case (sum,cnt) => (sum/cnt, cnt)}).
    join(rddMoviesKV).
    map({case (m,((r,cnt),t)) => (m,t,r,cnt)}).
    coalesce(1).
    toDF().write.format("csv").mode(SaveMode.Overwrite).save(path_output_avgRatPerMovie)

//avgRatPerMovie.toDebugString

rddMoviesKV: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[33] at map at <console>:31
avgRatPerMovie: Unit = ()


### Aggregate-and-BroadcastJoin

In [10]:
val rddMoviesKV = rddMovies.map(x => (x._1,x._2))
val bRddMovies = sc.broadcast(rddMoviesKV.collectAsMap())
val avgRatPerMovie = rddRatings.
    map(x => ((x._2),(x._3))).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).
    mapValues({case (sum,cnt) => (sum/cnt, cnt)}).
    map({case (m,(r,cnt)) => (m,bRddMovies.value.get(m),r,cnt)}).
    coalesce(1).
    toDF().write.format("csv").mode(SaveMode.Overwrite).save(path_output_avgRatPerMovie)

rddMoviesKV: org.apache.spark.rdd.RDD[(Long, String)] = MapPartitionsRDD[45] at map at <console>:32
bRddMovies: org.apache.spark.broadcast.Broadcast[scala.collection.Map[Long,String]] = Broadcast(15)
avgRatPerMovie: Unit = ()


## 104-3 Compute the average rating for each genre

Two possible workflows:

1. Pre-aggregation (3 shuffles)

  - Aggregate ratings by movieId
  - Join with movies and map to genres
  - Aggregate by genres
  
2. Join & aggregate (2 shuffles)

  - Join with movies and map to genres
  - Aggregate by genres

In [ ]:
val path_output_avgRatPerGenre = "s3a://"+bucketname+"/spark/avgRatPerGenre"

for ((k,v) <- sc.getPersistentRDDs) {
  v.unpersist()
}